### 팀명 : Versatile
### 팀원 : 강지민, 박소연
### 주제명 : 투자, 건강한 문화가 되다
### 분류 목적 및 방식 : 빠르게 변화할 수 있는 개인의 투자성향을 반영하기 위해 매월말을 기준으로 데이터를 갱신하여 고객을 분류한다

# 데이터 전처리

## 필요한 패키지 로드 및 데이터 로드

In [3]:
# 필요한 패키지 설치

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import seaborn as sns
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from matplotlib import rc
from yellowbrick.cluster import KElbowVisualizer
import math
import zipfile
import warnings
warnings.filterwarnings('ignore')
rc('font', family='NanumBarunGothic')
import re

import FinanceDataReader as fdr
from pykrx import stock
import pandas_datareader.data as pdr
import yfinance as yf

In [3]:
# 데이터 불러오기

info = pd.read_csv('/home/public/cus_ifo.csv',encoding='utf-8')
kr = pd.read_csv('/home/public/kr_stk_orr.csv',encoding='utf-8')
oss = pd.read_csv('/home/public/oss_stk_orr.csv',encoding='utf-8')
ast = pd.read_csv('/home/public/cus_tot_aet_ifo.csv',encoding='utf-8')
bnc = pd.read_csv('/home/public/cus_itg_sct_bnc.csv',encoding='utf-8')
GA = pd.read_csv('/home/public/GA.csv',encoding='utf-8',delimiter='|')

## 데이터 수집 및 전처리

### 1. 월말 잔고 데이터 분리 
- stocks/ETF, ETN/ELW

In [33]:
# 잔고데이터에 있는 주식, etf, 선물 및 모든 종목코드 불러오기
bnc_code = list(bnc['iem_cd'].unique()) 

#### - 국내

In [35]:
# 그 중 한국 종목코드 
bnc_kr_code = list(bnc[bnc['cur_cd']=='KRW'].iem_cd.unique())

#### 

In [38]:
#  잔고 국내 데이터에 ( ETN / ELW ) 있는지 확인.

num_start = list() 

for a in bnc_kr_code : 
    if re.match('[0-9]',a):
        num_start.append(a)

In [37]:
num_start # 숫자로 시작하는 코드가 (ETN/ELW) 없다

[]

In [41]:
start_A = list()

for a in bnc_kr_code : 
    if a.startswith('A') : 
        start_A.append(a)

In [42]:
start_A_k # k로 끝나는 코드가 (우선주)없다

[]

In [43]:
start_A_k = list()

for a in start_A : 
    if a.endswith('K') : 
        start_A_k.append(a)

In [45]:
start_A_c = list()

for a in start_A : 
    if a.endswith('C') : 
        start_A_c.append(a)

In [46]:
start_A_c 

[]

In [47]:
# 잔고 데이터의 국내 stocks, etf
bnc_stocks_etf = pd.DataFrame(start_A)

In [48]:
bnc_stocks_etf.rename(columns={0:'Code'},inplace=True)

In [49]:
#계좌 잔고 정보에 있는 주식,etf들은 bnc_stocks,etf로 저장함.
bnc_stocks_etf.to_csv('bnc_stocks_etf.csv',index = False)

In [36]:
# 국내 etf
# 잔고데이터 미리 분류해 놓음.

etf_bnc = pd.read_csv('etf_bnc.csv') 
etf_bnc

,code
0,A271050
1,A122630
2,A251340
3,A233740
4,A261220
...,...
182,A196230
183,A266360
184,A105780
185,A412560


In [50]:
# 잔고데이터의 ETN

start_notA = list()

for a in bnc_kr_code : 
    if a.startswith('A') : 
        pass
    else:
        start_notA.append(a.strip())

In [51]:
bnc_ETN = start_notA

In [53]:
bnc_ETN = pd.DataFrame(bnc_ETN)

In [54]:
bnc_ETN.to_csv('bnc_ETN.csv',index=False)

### 2. 국내 및 해외 일별 시세 데이터 불러오기
- finance datareader 패키지 사용.

#### <국내>

In [55]:
stocks_fdr = fdr.StockListing('KRX')
stocks_fdr

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,61000,2,-400,-0.65,61400,61700,60800,6677933,407953216200,364156735550000,5969782550,STK
1,373220,KR7373220003,LG에너지솔루션,KOSPI,,571000,1,3000,0.53,568000,577000,566000,189536,108270903000,133614000000000,234000000,STK
2,207940,KR7207940008,삼성바이오로직스,KOSPI,,903000,1,7000,0.78,900000,904000,895000,31687,28512433000,64270122000000,71174000,STK
3,000660,KR7000660001,SK하이닉스,KOSPI,,85100,2,-1800,-2.07,87200,87300,85100,1439011,123372985900,61953001261500,728002365,STK
4,051910,KR7051910008,LG화학,KOSPI,,721000,2,-7000,-0.96,726000,731000,718000,101478,73340712000,50897079303000,70592343,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,267810,KR7267810000,앙츠,KONEX,일반기업부,1160,2,-180,-13.43,1430,1430,1160,2,2590,2072920000,1787000,KNX
2672,288490,KR7288490006,나라소프트,KONEX,일반기업부,53,1,2,3.92,52,58,48,58197,3137902,2021791000,38147000,KNX
2673,215050,KR7215050006,비엔디생활건강,KONEX,일반기업부,912,0,0,0.00,0,0,0,0,0,1934959392,2121666,KNX
2674,267060,KR7267060002,명진홀딩스,KONEX,일반기업부,181,0,0,0.00,0,0,0,0,0,1654666524,9141804,KNX


In [56]:
# 2201~2206 기간동안 상폐된 종목들 확인 후 'delete_01_06'으로 저장¶

krx_delisting = fdr.StockListing('KRX-DELISTING')
krx_delisting

,Symbol,Name,Market,SecuGroup,Kind,ListingDate,DelistingDate,Reason,ArrantEnforceDate,ArrantEndDate,Industry,ParValue,ListingShares,ToSymbol,ToName
0,06031012,3S R,KOSDAQ,신주인수권증서,보통주,2012-05-14,2012-05-21,,NaT,NaT,,0.0,1194422.0,,
1,00684014,AK홀딩스8R,KOSPI,신주인수권증서,보통주,2014-07-28,2014-08-04,,NaT,NaT,,0.0,1278299.0,,
2,13893015,BNK금융지주 8R,KOSPI,신주인수권증서,보통주,2015-12-24,2016-01-05,,NaT,NaT,,0.0,55969410.0,,
3,13893014,BS금융지주5R,KOSPI,신주인수권증서,보통주,2014-06-18,2014-06-25,,NaT,NaT,,0.0,32791220.0,,
4,03204017,C&S자산관리 34R,KOSDAQ,신주인수권증서,보통주,2017-02-02,2017-02-09,,NaT,NaT,,0.0,3995063.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3454,034370,럭키소재,KOSPI,주권,,1979-06-23,1991-11-11,해산 사유 발생,NaT,NaT,,NaN,NaN,,
3455,028460,태평양건설,KOSPI,주권,,1977-06-09,1991-10-05,영업활동정지 6월 계속,1991-08-28,1991-10-04,,NaN,NaN,,
3456,028450,금성투자금융,KOSPI,주권,,1986-08-29,1991-09-02,해산 사유 발생,NaT,NaT,,NaN,NaN,,
3457,028440,삼화,KOSPI,주권,,1974-06-12,1991-07-12,감사의견 의견거절,1991-06-05,1991-07-11,,NaN,NaN,,


In [57]:
krx_delisting.DelistingDate = krx_delisting.DelistingDate.astype('string')

In [58]:
delete_01 = krx_delisting[krx_delisting['DelistingDate'].str.contains('2022-01')]
delete_02 = krx_delisting[krx_delisting['DelistingDate'].str.contains('2022-02')]
delete_03 = krx_delisting[krx_delisting['DelistingDate'].str.contains('2022-03')]
delete_04 = krx_delisting[krx_delisting['DelistingDate'].str.contains('2022-04')]
delete_05 = krx_delisting[krx_delisting['DelistingDate'].str.contains('2022-05')]
delete_06 = krx_delisting[krx_delisting['DelistingDate'].str.contains('2022-06')]

In [59]:
delete_01_06 = pd.concat([delete_01,delete_02,delete_03,delete_04,delete_05,delete_06])

In [60]:
delete_01_06.to_csv('delete_01_06.csv',index = False)

In [61]:
delete_01_06

,Symbol,Name,Market,SecuGroup,Kind,ListingDate,DelistingDate,Reason,ArrantEnforceDate,ArrantEndDate,Industry,ParValue,ListingShares,ToSymbol,ToName
110,2903801C,대유 7R,KOSDAQ,신주인수권증서,보통주,2022-01-21,2022-01-28,,NaT,NaT,,0.0,7992590.0,,
148,0340201C,두산중공업 44R,KOSPI,신주인수권증서,보통주,2022-01-21,2022-01-28,,NaT,NaT,,0.0,66063103.0,,
782,101060,SBS미디어홀딩스,KOSPI,주권,보통주,2008-03-24,2022-01-18,해산 사유 발생,NaT,NaT,서비스업,500.0,139896190.0,,
783,015350,부산가스,KOSPI,주권,보통주,1997-06-23,2022-01-18,지주회사(최대주주등)의 완전자회사화 등,NaT,NaT,전기가스업,5000.0,11000000.0,,
784,221610,자안바이오,KOSDAQ,주권,보통주,2016-01-27,2022-01-05,발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 ...,2021-12-24,2022-01-04,화학,5000.0,6654797.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745,216280,원텍,KONEX,주권,보통주,2015-04-29,2022-06-30,피흡수합병,NaT,NaT,-,500.0,6330040.0,,
746,00004217,KR모터스 1WR,KOSPI,신주인수권증권,보통주,2017-08-14,2022-06-29,신주인수권증권 행사기간 및 행사의 만료,NaT,NaT,,0.0,30439305.0,000040,KR모터스
747,176440,에이치엔티,KOSDAQ,주권,보통주,2016-11-28,2022-06-27,기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정,2022-06-16,2022-06-24,IT부품,100.0,85739345.0,,
748,00306217,에이프로젠제약 13WR,KOSPI,신주인수권증권,보통주,2017-08-09,2022-06-21,신주인수권증권 행사기간 및 행사의 만료,NaT,NaT,,0.0,12181109.0,003060,에이프로젠제약


In [62]:
# Code_fdr = fdr 에서 불러온 주식, etf 국내 종목코드. 
# 2201~2206사이 상장폐지 된 종목포함.

Code_fdr = list(stocks_fdr.Code) + list(delete_01_06.Symbol) + list(etf_list_fdr.Symbol)

In [63]:
len(Code_fdr) # 3391개

3391

In [73]:
stocks_fdr_m = stocks_fdr[['Code','Name']]
delete_01_06_m = delete_01_06[['Symbol','Name']]

In [80]:
delete_01_06_m.rename(columns={'Symbol':'Code'},inplace=True)
etf_list_fdr.rename(columns={'Symbol':'Code'},inplace=True)

In [83]:
# fdr에서 불러온 stock, etf, 상폐 종목 concat.

fdr_krx = pd.concat([stocks_fdr_m, etf_list_fdr ,delete_01_06_m])

In [84]:
fdr_krx

,Code,Name
0,005930,삼성전자
1,373220,LG에너지솔루션
2,207940,삼성바이오로직스
3,000660,SK하이닉스
4,051910,LG화학
...,...,...
745,216280,원텍
746,00004217,KR모터스 1WR
747,176440,에이치엔티
748,00306217,에이프로젠제약 13WR


- fdr에서 국내 데이터 불러오기

In [ ]:
# 여기 코드는 돌리는데 시간이 걸립니다.
# 결과  = 'fdr_11_06.csv'

df_fdr = pd.DataFrame()

for a in Code_fdr :

    start_date = '2021-10-29'
    end_date = '2022-06-30'
 
    df = fdr.DataReader( a , start=start_date, end=end_date)
    df['Code'] = a
    df_fdr = pd.concat([df, df_fdr])

In [ ]:
df_fdr = df_fdr.reset_index().merge(fdr_krx , how='left', on = 'Code')

In [ ]:
# 앞에 문자 추가해서 저장해줘야 0 안사라짐.
# df_fdr에는 주식과, ETF 만 있으므로 'A'추가.

df_fdr.Code = 'A' + df_fdr.Code 

In [ ]:
df_fdr.rename(columns={'index':'Date'},inplace=True)

In [ ]:
# 이후 사용의 편리함을 위해 년월로만 표기된 열 추가
df_fdr['Date_m'] = df_fdr['Date'].dt.strftime('%Y-%m')

In [4]:
# 위 데이터 불러오는 코드가 시간이 오래 걸리므로 미리 저장해놓았습니다.

df_fdr = pd.read_csv('fdr_11_06.csv')

In [5]:
df_fdr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520876 entries, 0 to 520875
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Date        520876 non-null  object 
 1   Open        520876 non-null  float64
 2   High        520876 non-null  float64
 3   Low         520876 non-null  float64
 4   Close       520876 non-null  float64
 5   Volume      520876 non-null  float64
 6   Change      520717 non-null  float64
 7   Code        520876 non-null  object 
 8   Name        520876 non-null  object 
 9   Change_cal  520875 non-null  float64
 10  Date_m      520876 non-null  object 
dtypes: float64(7), object(4)
memory usage: 43.7+ MB


- 코스피 지수 불러오기

In [12]:
# 코스피 지수 불러오기 
df_kospi = fdr.DataReader('KS11','2021-10-29','2022-06-30')

In [13]:
df_kospi.reset_index(inplace=True)

In [14]:
df_kospi.to_csv('df_kospi.csv',index=False)

#### <해외>

In [51]:
oss_ticker = pd.read_csv('oss_isin_ticker.csv')

In [52]:
oss_ticker.symbol = oss_ticker.symbol.astype('string')

In [53]:
oss_ticker.dropna(inplace=True)

In [54]:
oss_ticker.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226 entries, 0 to 1276
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   iem_cd  1226 non-null   object
 1   cur_cd  1226 non-null   object
 2   symbol  1226 non-null   string
dtypes: object(2), string(1)
memory usage: 38.3+ KB


In [55]:
code_oss = list(oss_ticker.symbol)

#### - 해외 데이터 불러오기 위한 for 문

In [ ]:
code_oss = list(oss_ticker.symbol)

In [ ]:
## yfinance로 해외데이터 불러오기 (다소 오래걸림..)
## 결과 : oss_fdr

oss_fdr = pd.DataFrame()

for a in code_oss :
    yf.download(a, start=start_date, end=end_date)
    df['Code'] = a
    oss_fdr = pd.concat([df, oss_fdr])

In [ ]:
oss_ticker.rename(columns={'symbol':'Code'},inplace=True)

In [ ]:
oss_yf = oss_fdr.reset_index().merge(oss_ticker, how='left',on='Code')

In [ ]:
# 이후 편의를 위해 년월에 대한 열 추가.

oss_yf['Date_m'] = oss_yf['Date'].dt.strftime('%Y-%m')

- S&P 500

In [56]:
Snp500 = yf.download('^GSPC', start='2021-10-29', end='2022-06-30')

[*********************100%***********************]  1 of 1 completed


In [57]:
Snp500.reset_index(inplace=True)

In [58]:
Snp500

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-10-29,4572.870117,4608.080078,4567.589844,4605.379883,4605.379883,4510200000
1,2021-11-01,4610.620117,4620.339844,4595.060059,4613.669922,4613.669922,3971540000
2,2021-11-02,4613.339844,4635.149902,4613.339844,4630.649902,4630.649902,3975250000
3,2021-11-03,4630.649902,4663.459961,4621.189941,4660.569824,4660.569824,4319660000
4,2021-11-04,4662.930176,4683.000000,4662.589844,4680.060059,4680.060059,4462300000
...,...,...,...,...,...,...,...
162,2022-06-23,3774.709961,3802.580078,3743.520020,3795.729980,3795.729980,5098640000
163,2022-06-24,3821.750000,3913.649902,3821.750000,3911.739990,3911.739990,8120260000
164,2022-06-27,3920.760010,3927.719971,3889.659912,3900.110107,3900.110107,4325310000
165,2022-06-28,3913.000000,3945.860107,3820.139893,3821.550049,3821.550049,4270120000


## 변수 생성
1. 포트폴리오 베타
2. 월평균 일일 거래 빈도
3. 우량주 투자 비율
4. 파생상품 보유 개수 (국내만)
5. 정보 수집 성향 지수

### 종목별 가중치 계산

In [ ]:
# 계좌/고객정보년월/종목코드에 따라 결제기준 현재평가 금액 합계 계산 

## 종목당 투자금액 확인
bnc_grouped = bnc.groupby(['act_no', 'bse_ym', 'iem_cd'])[['stl_bse_now_eal_amt']].sum()
bnc_grouped

In [ ]:
# 고객정보년월당 투자금액 확인 (계좌 하나당 월별 주식 잔고 합계 확인)
bnc_grouped.sum(axis =0, level= [0,1])

In [ ]:
# dist (개별 종목에 대한 투자 비율) = (개별 종목 투자 금액) / (보유 종목들에 대한 총 투자 금액)
bnc_grouped['dist'] = bnc_grouped.sum(axis=0, level=[0,1,2])/bnc_grouped.sum(axis =0, level= [0,1] )
bnc_grouped = bnc_grouped.fillna(0)

In [ ]:
bnc_grouped

In [ ]:
bnc_grouped.reset_index(inplace = True)
bnc_grouped = bnc_grouped.rename(columns = {'stl_bse_now_eal_amt' : 'tot_amt' })
bnc_grouped

In [ ]:
bnc_grouped.to_csv('bnc_dist.csv', index= False)

### 1) 포트폴리오 베타 (월 말일 기준)

#### < 국내 >

- stock/ETF

In [6]:
df_fdr_1 = df_fdr[df_fdr['Date'].between('2021-10-28', '2022-01-31')]
df_fdr_2 = df_fdr[df_fdr['Date'].between('2021-11-29', '2022-02-28')]
df_fdr_3 = df_fdr[df_fdr['Date'].between('2021-12-29', '2022-03-31')]
df_fdr_4 = df_fdr[df_fdr['Date'].between('2022-01-28', '2022-04-30')]
df_fdr_5 = df_fdr[df_fdr['Date'].between('2022-02-26', '2022-05-31')]
df_fdr_6 = df_fdr[df_fdr['Date'].between('2022-03-28', '2022-06-30')]


In [7]:
# 등락률 구하기
df_fdr_1['Change_cal'] = (df_fdr_1['Close'] - df_fdr_1['Close'].shift(1)) / df_fdr_1['Close'].shift(1) * 100
df_fdr_2['Change_cal'] = (df_fdr_2['Close'] - df_fdr_2['Close'].shift(1)) / df_fdr_2['Close'].shift(1) * 100
df_fdr_3['Change_cal'] = (df_fdr_3['Close'] - df_fdr_3['Close'].shift(1)) / df_fdr_3['Close'].shift(1) * 100
df_fdr_4['Change_cal'] = (df_fdr_4['Close'] - df_fdr_4['Close'].shift(1)) / df_fdr_4['Close'].shift(1) * 100
df_fdr_5['Change_cal'] = (df_fdr_5['Close'] - df_fdr_5['Close'].shift(1)) / df_fdr_5['Close'].shift(1) * 100
df_fdr_6['Change_cal'] = (df_fdr_6['Close'] - df_fdr_6['Close'].shift(1)) / df_fdr_6['Close'].shift(1) * 100

In [8]:
dfr_1 = df_fdr_1[df_fdr_1['Date_m'].between('2021-11', '2022-01')]
dfr_2 = df_fdr_2[df_fdr_2['Date_m'].between('2021-12', '2022-02')]
dfr_3 = df_fdr_3[df_fdr_3['Date_m'].between('2022-01', '2022-03')]
dfr_4 = df_fdr_4[df_fdr_4['Date_m'].between('2022-02', '2022-04')]
dfr_5 = df_fdr_5[df_fdr_5['Date_m'].between('2022-03', '2022-05')]
dfr_6 = df_fdr_6[df_fdr_6['Date_m'].between('2022-04', '2022-06')]


In [9]:
grouped1 = dfr_1.groupby(['Code'])
grouped2 = dfr_2.groupby(['Code'])
grouped3 = dfr_3.groupby(['Code'])
grouped4 = dfr_4.groupby(['Code'])
grouped5 = dfr_5.groupby(['Code'])
grouped6 = dfr_6.groupby(['Code'])

In [10]:
# 개별종목의 표준편차 구하기.

dfr_1['stock_std'] = grouped1.Change_cal.transform('std')
dfr_2['stock_std'] = grouped2.Change_cal.transform('std')
dfr_3['stock_std'] = grouped3.Change_cal.transform('std')
dfr_4['stock_std'] = grouped4.Change_cal.transform('std')
dfr_5['stock_std'] = grouped5.Change_cal.transform('std')
dfr_6['stock_std'] = grouped6.Change_cal.transform('std')

In [11]:
# 나눠준 파일 모두 저장
dfr_1.to_csv('dfr_1.csv',index=False)
dfr_2.to_csv('dfr_2.csv',index=False)
dfr_3.to_csv('dfr_3.csv',index=False)
dfr_4.to_csv('dfr_4.csv',index=False)
dfr_5.to_csv('dfr_5.csv',index=False)
dfr_6.to_csv('dfr_6.csv',index=False)

- Kospi

In [15]:
# kospi 등락률 구하기
df_kospi['Change_cal'] = (df_kospi['Adj Close'] - df_kospi['Adj Close'].shift(1)) / df_kospi['Adj Close'].shift(1) * 100

In [16]:
df_kospi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        163 non-null    datetime64[ns]
 1   Open        163 non-null    float64       
 2   High        163 non-null    float64       
 3   Low         163 non-null    float64       
 4   Close       163 non-null    float64       
 5   Adj Close   163 non-null    float64       
 6   Volume      163 non-null    int64         
 7   Change_cal  162 non-null    float64       
dtypes: datetime64[ns](1), float64(6), int64(1)
memory usage: 10.3 KB


In [17]:
df_kospi.Date = pd.to_datetime(df_kospi.Date)

In [18]:
df_kospi['Date_m'] = df_kospi['Date'].dt.strftime('%Y-%m')

kospi_1 = df_kospi[df_kospi['Date_m'].between('2021-11', '2022-01')]
kospi_2 = df_kospi[df_kospi['Date_m'].between('2021-12', '2022-02')]
kospi_3 = df_kospi[df_kospi['Date_m'].between('2022-01', '2022-03')]
kospi_4 = df_kospi[df_kospi['Date_m'].between('2022-02', '2022-04')]
kospi_5 = df_kospi[df_kospi['Date_m'].between('2022-03', '2022-05')]
kospi_6 = df_kospi[df_kospi['Date_m'].between('2022-04', '2022-06')]


In [19]:
# 코스피의 표준편차 구하기.

kospi_1_std = kospi_1.Change_cal.std()
kospi_2_std = kospi_2.Change_cal.std()
kospi_3_std = kospi_3.Change_cal.std()
kospi_4_std = kospi_4.Change_cal.std()
kospi_5_std = kospi_5.Change_cal.std()
kospi_6_std = kospi_6.Change_cal.std()

In [20]:
kospi_1['kospi_std'] = kospi_1_std
kospi_2['kospi_std'] = kospi_2_std
kospi_3['kospi_std'] = kospi_3_std
kospi_4['kospi_std'] = kospi_4_std
kospi_5['kospi_std'] = kospi_5_std
kospi_6['kospi_std'] = kospi_6_std

-  stock, etf, kospi 모두 합쳐진 데이터프레임 만들기

In [21]:
dfr_1_1 = dfr_1[['Date','Change_cal','Code','stock_std']]
dfr_2_2 = dfr_2[['Date','Change_cal','Code','stock_std']]
dfr_3_3 = dfr_3[['Date','Change_cal','Code','stock_std']]
dfr_4_4 = dfr_4[['Date','Change_cal','Code','stock_std']]
dfr_5_5 = dfr_5[['Date','Change_cal','Code','stock_std']]
dfr_6_6 = dfr_6[['Date','Change_cal','Code','stock_std']]


In [23]:
kospi_1_1 = kospi_1[['Date','Change_cal','kospi_std']]
kospi_2_2 = kospi_2[['Date','Change_cal','kospi_std']]
kospi_3_3 = kospi_3[['Date','Change_cal','kospi_std']]
kospi_4_4 = kospi_4[['Date','Change_cal','kospi_std']]
kospi_5_5 = kospi_5[['Date','Change_cal','kospi_std']]
kospi_6_6 = kospi_6[['Date','Change_cal','kospi_std']]

In [24]:
dfr_1_1.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
kospi_1_1.rename(columns={'Change_cal':'Change_kospi'},inplace=True)
dfr_2_2.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
kospi_2_2.rename(columns={'Change_cal':'Change_kospi'},inplace=True)
dfr_3_3.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
kospi_3_3.rename(columns={'Change_cal':'Change_kospi'},inplace=True)
dfr_4_4.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
kospi_4_4.rename(columns={'Change_cal':'Change_kospi'},inplace=True)
dfr_5_5.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
kospi_5_5.rename(columns={'Change_cal':'Change_kospi'},inplace=True)
dfr_6_6.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
kospi_6_6.rename(columns={'Change_cal':'Change_kospi'},inplace=True)

In [25]:
dfr_1_1.Date = pd.to_datetime(dfr_1_1.Date)
dfr_2_2.Date = pd.to_datetime(dfr_2_2.Date)
dfr_3_3.Date = pd.to_datetime(dfr_3_3.Date)
dfr_4_4.Date = pd.to_datetime(dfr_4_4.Date)
dfr_5_5.Date = pd.to_datetime(dfr_5_5.Date)
dfr_6_6.Date = pd.to_datetime(dfr_6_6.Date)

In [28]:
forbeta1 = pd.merge(dfr_1_1,kospi_1_1,how='left',on='Date')
forbeta2 = pd.merge(dfr_2_2,kospi_2_2,how='left',on='Date')
forbeta3 = pd.merge(dfr_3_3,kospi_3_3,how='left',on='Date')
forbeta4 = pd.merge(dfr_4_4,kospi_4_4,how='left',on='Date')
forbeta5 = pd.merge(dfr_5_5,kospi_5_5,how='left',on='Date')
forbeta6 = pd.merge(dfr_6_6,kospi_6_6,how='left',on='Date')

In [29]:
#kospi 데이터 불러오는 과정 중, 
#2022-01-03, 2022-05-09, 2022-06-30 데이터가 누락되어 해당 날 행 제거.

forbeta1 = forbeta1.dropna(axis=0)
forbeta2 = forbeta2.dropna(axis=0)
forbeta3 = forbeta3.dropna(axis=0)
forbeta4 = forbeta4.dropna(axis=0)
forbeta5 = forbeta5.dropna(axis=0)
forbeta6 = forbeta6.dropna(axis=0)

In [30]:
forbeta1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195743 entries, 0 to 198863
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Date          195743 non-null  datetime64[ns]
 1   Change_stock  195743 non-null  float64       
 2   stock_code    195743 non-null  object        
 3   stock_std     195743 non-null  float64       
 4   Change_kospi  195743 non-null  float64       
 5   kospi_std     195743 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 10.5+ MB


In [31]:
forbeta1

,Date,Change_stock,stock_code,stock_std,Change_kospi,kospi_std
0,2021-11-01,-0.173595,A287320,1.442717,0.278051,1.092023
1,2021-11-02,2.068461,A287320,1.442717,1.159810,1.092023
2,2021-11-03,-1.631994,A287320,1.442717,-1.253697,1.092023
3,2021-11-04,0.091158,A287320,1.442717,0.252377,1.092023
4,2021-11-05,-0.765027,A287320,1.442717,-0.467614,1.092023
...,...,...,...,...,...,...
198859,2022-01-24,-0.661376,A005930,1.391445,-1.492086,1.092023
198860,2022-01-25,-1.464714,A005930,1.391445,-2.564832,1.092023
198861,2022-01-26,-0.945946,A005930,1.391445,-0.409864,1.092023
198862,2022-01-27,-2.728513,A005930,1.391445,-3.497291,1.092023


- 베타계수 구하기

In [32]:
# kospi 와 종목 간 상관계수 구하기
corr_stock_kospi = lambda x : x['Change_stock'].corr(x['Change_kospi'])

In [33]:
# null 값 존재, fdr 데이터 불러오기 오류.
# 모든 종가가 다 같은 값으로 불러와져 등락률이 0으로 나와 상관계수를 구할 수 없음. 

corr1 = pd.DataFrame(forbeta1.groupby('stock_code').apply(corr_stock_kospi))
corr2 = pd.DataFrame(forbeta2.groupby('stock_code').apply(corr_stock_kospi))
corr3 = pd.DataFrame(forbeta3.groupby('stock_code').apply(corr_stock_kospi))
corr4 = pd.DataFrame(forbeta4.groupby('stock_code').apply(corr_stock_kospi))
corr5 = pd.DataFrame(forbeta5.groupby('stock_code').apply(corr_stock_kospi))
corr6 = pd.DataFrame(forbeta6.groupby('stock_code').apply(corr_stock_kospi))

In [34]:
forbeta1_ = pd.merge(forbeta1,corr1,how='left',on='stock_code')
forbeta2_ = pd.merge(forbeta2,corr2,how='left',on='stock_code')
forbeta3_ = pd.merge(forbeta3,corr3,how='left',on='stock_code')
forbeta4_ = pd.merge(forbeta4,corr4,how='left',on='stock_code')
forbeta5_ = pd.merge(forbeta5,corr5,how='left',on='stock_code')
forbeta6_ = pd.merge(forbeta6,corr6,how='left',on='stock_code')

In [36]:
forbeta1_.rename(columns={0:'corr'},inplace=True)
forbeta2_.rename(columns={0:'corr'},inplace=True)
forbeta3_.rename(columns={0:'corr'},inplace=True)
forbeta4_.rename(columns={0:'corr'},inplace=True)
forbeta5_.rename(columns={0:'corr'},inplace=True)
forbeta6_.rename(columns={0:'corr'},inplace=True)

In [37]:
forbeta1_ = forbeta1_.drop_duplicates(['stock_code'])
forbeta2_ = forbeta2_.drop_duplicates(['stock_code'])
forbeta3_ = forbeta3_.drop_duplicates(['stock_code'])
forbeta4_ = forbeta4_.drop_duplicates(['stock_code'])
forbeta5_ = forbeta5_.drop_duplicates(['stock_code'])
forbeta6_ = forbeta6_.drop_duplicates(['stock_code'])

In [38]:
forbeta1_.to_csv('forbeta1_kr.csv',index=False)
forbeta2_.to_csv('forbeta2_kr.csv',index=False)
forbeta3_.to_csv('forbeta3_kr.csv',index=False)
forbeta4_.to_csv('forbeta4_kr.csv',index=False)
forbeta5_.to_csv('forbeta5_kr.csv',index=False)
forbeta6_.to_csv('forbeta6_kr.csv',index=False)

In [39]:
# 베타 생성 함수 만들기
# (개별주식 표준편차 * 시장 표준편차 * 둘의 상관계수)/ (시장 표준편차^2)

def beta(a,b,c):
    return (a*b*c)/(b**2)

In [40]:
forbeta1_['beta'] = forbeta1_.apply(lambda x : beta(x['corr'],x['kospi_std'],x['stock_std']), axis=1)
forbeta2_['beta'] = forbeta2_.apply(lambda x : beta(x['corr'],x['kospi_std'],x['stock_std']), axis=1)
forbeta3_['beta'] = forbeta3_.apply(lambda x : beta(x['corr'],x['kospi_std'],x['stock_std']), axis=1)
forbeta4_['beta'] = forbeta4_.apply(lambda x : beta(x['corr'],x['kospi_std'],x['stock_std']), axis=1)
forbeta5_['beta'] = forbeta5_.apply(lambda x : beta(x['corr'],x['kospi_std'],x['stock_std']), axis=1)
forbeta6_['beta'] = forbeta6_.apply(lambda x : beta(x['corr'],x['kospi_std'],x['stock_std']), axis=1)

In [41]:
# fdr로 구한 데이터 베타는 다 구함.
forbeta1_.to_csv('fdr_beta_1_kr.csv',index=False)
forbeta2_.to_csv('fdr_beta_2_kr.csv',index=False)
forbeta3_.to_csv('fdr_beta_3_kr.csv',index=False)
forbeta4_.to_csv('fdr_beta_4_kr.csv',index=False)
forbeta5_.to_csv('fdr_beta_5_kr.csv',index=False)
forbeta6_.to_csv('fdr_beta_6_kr.csv',index=False)

In [42]:
forbeta1_['bse_ym']='202201'
forbeta2_['bse_ym']='202202'
forbeta3_['bse_ym']='202203'
forbeta4_['bse_ym']='202204'
forbeta5_['bse_ym']='202205'
forbeta6_['bse_ym']='202206'

In [43]:
fdr_beta_01_06 = pd.concat([forbeta1_,forbeta2_,forbeta3_,forbeta4_,forbeta5_,forbeta6_])

In [44]:
fdr_beta_01_06 = fdr_beta_01_06[['bse_ym','stock_code','beta']]

In [46]:
# 베타계수 결측은 0으로 처리.
fdr_beta_01_06.beta = fdr_beta_01_06.beta.fillna(0)

In [47]:
fdr_beta_01_06.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19015 entries, 0 to 189936
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   bse_ym      19015 non-null  object 
 1   stock_code  19015 non-null  object 
 2   beta        19015 non-null  float64
dtypes: float64(1), object(2)
memory usage: 594.2+ KB


In [48]:
fdr_beta_01_06.to_csv('kr_fdr_beta_all.csv',index = False)

In [49]:
kr_fdr_beta_all = pd.read_csv('kr_fdr_beta_all.csv')

#### < 해외 >

In [ ]:
# 90일 베타계수를 구하기 위해 데이터프레임 분리.
oss_yf_1 = oss_yf[oss_yf['Date'].between('2021-10-28', '2022-01-31')]
oss_yf_2 = oss_yf[oss_yf['Date'].between('2021-11-29', '2022-02-28')]
oss_yf_3 = oss_yf[oss_yf['Date'].between('2021-12-29', '2022-03-31')]
oss_yf_4 = oss_yf[oss_yf['Date'].between('2022-01-28', '2022-04-30')]
oss_yf_5 = oss_yf[oss_yf['Date'].between('2022-02-26', '2022-05-31')]
oss_yf_6 = oss_yf[oss_yf['Date'].between('2022-03-28', '2022-06-30')]


In [ ]:
# 등락률 구하기
oss_yf_1['Change_cal'] = (oss_yf_1['Adj Close'] - oss_yf_1['Adj Close'].shift(1)) / oss_yf_1['Adj Close'].shift(1) * 100
oss_yf_2['Change_cal'] = (oss_yf_2['Adj Close'] - oss_yf_2['Adj Close'].shift(1)) / oss_yf_2['Adj Close'].shift(1) * 100
oss_yf_3['Change_cal'] = (oss_yf_3['Adj Close'] - oss_yf_3['Adj Close'].shift(1)) / oss_yf_3['Adj Close'].shift(1) * 100
oss_yf_4['Change_cal'] = (oss_yf_4['Adj Close'] - oss_yf_4['Adj Close'].shift(1)) / oss_yf_4['Adj Close'].shift(1) * 100
oss_yf_5['Change_cal'] = (oss_yf_5['Adj Close'] - oss_yf_5['Adj Close'].shift(1)) / oss_yf_5['Adj Close'].shift(1) * 100
oss_yf_6['Change_cal'] = (oss_yf_6['Adj Close'] - oss_yf_6['Adj Close'].shift(1)) / oss_yf_6['Adj Close'].shift(1) * 100

In [ ]:
oss_yf_1 = oss_yf_1[oss_yf_1['Date_m'].between('2021-11', '2022-01')]
oss_yf_2 = oss_yf_2[oss_yf_2['Date_m'].between('2021-12', '2022-02')]
oss_yf_3 = oss_yf_3[oss_yf_3['Date_m'].between('2022-01', '2022-03')]
oss_yf_4 = oss_yf_4[oss_yf_4['Date_m'].between('2022-02', '2022-04')]
oss_yf_5 = oss_yf_5[oss_yf_5['Date_m'].between('2022-03', '2022-05')]
oss_yf_6 = oss_yf_6[oss_yf_6['Date_m'].between('2022-04', '2022-06')]


In [ ]:
grouped1 = oss_yf_1.groupby(['Code'])
grouped2 = oss_yf_2.groupby(['Code'])
grouped3 = oss_yf_3.groupby(['Code'])
grouped4 = oss_yf_4.groupby(['Code'])
grouped5 = oss_yf_5.groupby(['Code'])
grouped6 = oss_yf_6.groupby(['Code'])

In [ ]:
# 개별종목의 표준편차 구하기.

oss_yf_1['stock_std'] = grouped1.Change_cal.transform('std')
oss_yf_2['stock_std'] = grouped2.Change_cal.transform('std')
oss_yf_3['stock_std'] = grouped3.Change_cal.transform('std')
oss_yf_4['stock_std'] = grouped4.Change_cal.transform('std')
oss_yf_5['stock_std'] = grouped5.Change_cal.transform('std')
oss_yf_6['stock_std'] = grouped6.Change_cal.transform('std')

In [ ]:
# 나눠준 파일 다 저장

oss_yf_1.to_csv('oss_yf_1.csv',index = False)
oss_yf_2.to_csv('oss_yf_2.csv',index = False)
oss_yf_3.to_csv('oss_yf_3.csv',index = False)
oss_yf_4.to_csv('oss_yf_4.csv',index = False)
oss_yf_5.to_csv('oss_yf_5.csv',index = False)
oss_yf_6.to_csv('oss_yf_6.csv',index = False)

- S&P 500

In [ ]:
# S&P500 등락률 구하기
Snp500['Change_cal'] = (Snp500['Adj Close'] - Snp500['Adj Close'].shift(1)) / Snp500['Adj Close'].shift(1) * 100

In [ ]:
Snp500['Date_m'] = Snp500['Date'].dt.strftime('%Y-%m')

Snp500_1 = Snp500[Snp500['Date_m'].between('2021-11', '2022-01')]
Snp500_2 = Snp500[Snp500['Date_m'].between('2021-12', '2022-02')]
Snp500_3 = Snp500[Snp500['Date_m'].between('2022-01', '2022-03')]
Snp500_4 = Snp500[Snp500['Date_m'].between('2022-02', '2022-04')]
Snp500_5 = Snp500[Snp500['Date_m'].between('2022-03', '2022-05')]
Snp500_6 = Snp500[Snp500['Date_m'].between('2022-04', '2022-06')]


In [ ]:
# S&P500의 표준편차 구하기.

Snp500_1_std = Snp500_1.Change_cal.std()
Snp500_2_std = Snp500_2.Change_cal.std()
Snp500_3_std = Snp500_3.Change_cal.std()
Snp500_4_std = Snp500_4.Change_cal.std()
Snp500_5_std = Snp500_5.Change_cal.std()
Snp500_6_std = Snp500_6.Change_cal.std()

In [ ]:
Snp500_1['Snp500_std'] = Snp500_1_std
Snp500_2['Snp500_std'] = Snp500_2_std
Snp500_3['Snp500_std'] = Snp500_3_std
Snp500_4['Snp500_std'] = Snp500_4_std
Snp500_5['Snp500_std'] = Snp500_5_std
Snp500_6['Snp500_std'] = Snp500_6_std

In [ ]:
oss_yf_1_1 = oss_yf_1[['Date','Code','iem_cd','cur_cd','Change_cal','stock_std']]
oss_yf_2_2 = oss_yf_2[['Date','Code','iem_cd','cur_cd','Change_cal','stock_std']]
oss_yf_3_3 = oss_yf_3[['Date','Code','iem_cd','cur_cd','Change_cal','stock_std']]
oss_yf_4_4 = oss_yf_4[['Date','Code','iem_cd','cur_cd','Change_cal','stock_std']]
oss_yf_5_5 = oss_yf_5[['Date','Code','iem_cd','cur_cd','Change_cal','stock_std']]
oss_yf_6_6 = oss_yf_6[['Date','Code','iem_cd','cur_cd','Change_cal','stock_std']]

In [ ]:
Snp500_1_1 = Snp500_1[['Date', 'Change_cal','Snp500_std']]
Snp500_2_2 = Snp500_2[['Date', 'Change_cal','Snp500_std']]
Snp500_3_3 = Snp500_3[['Date', 'Change_cal','Snp500_std']]
Snp500_4_4 = Snp500_4[['Date', 'Change_cal','Snp500_std']]
Snp500_5_5 = Snp500_5[['Date', 'Change_cal','Snp500_std']]
Snp500_6_6 = Snp500_6[['Date', 'Change_cal','Snp500_std']]

In [ ]:
oss_yf_1_1.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
Snp500_1_1.rename(columns={'Change_cal':'Change_snp500'},inplace=True)
oss_yf_2_2.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
Snp500_2_2.rename(columns={'Change_cal':'Change_snp500'},inplace=True)
oss_yf_3_3.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
Snp500_3_3.rename(columns={'Change_cal':'Change_snp500'},inplace=True)
oss_yf_4_4.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
Snp500_4_4.rename(columns={'Change_cal':'Change_snp500'},inplace=True)
oss_yf_5_5.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
Snp500_5_5.rename(columns={'Change_cal':'Change_snp500'},inplace=True)
oss_yf_6_6.rename(columns={'Change_cal':'Change_stock','Code':'stock_code'},inplace=True)
Snp500_6_6.rename(columns={'Change_cal':'Change_snp500'},inplace=True)

In [ ]:
forbeta1 = pd.merge(oss_yf_1_1,Snp500_1_1,how='left',on='Date')
forbeta2 = pd.merge(oss_yf_2_2,Snp500_2_2,how='left',on='Date')
forbeta3 = pd.merge(oss_yf_3_3,Snp500_3_3,how='left',on='Date')
forbeta4 = pd.merge(oss_yf_4_4,Snp500_4_4,how='left',on='Date')
forbeta5 = pd.merge(oss_yf_5_5,Snp500_5_5,how='left',on='Date')
forbeta6 = pd.merge(oss_yf_6_6,Snp500_6_6,how='left',on='Date')

In [ ]:
# S&P500 과 종목 간 상관계수 구하기
corr_stock_snp500 = lambda x : x['Change_stock'].corr(x['Change_snp500'])

In [ ]:
corr1 = pd.DataFrame(forbeta1.groupby('stock_code').apply(corr_stock_snp500))
corr2 = pd.DataFrame(forbeta2.groupby('stock_code').apply(corr_stock_snp500))
corr3 = pd.DataFrame(forbeta3.groupby('stock_code').apply(corr_stock_snp500))
corr4 = pd.DataFrame(forbeta4.groupby('stock_code').apply(corr_stock_snp500))
corr5 = pd.DataFrame(forbeta5.groupby('stock_code').apply(corr_stock_snp500))
corr6 = pd.DataFrame(forbeta6.groupby('stock_code').apply(corr_stock_snp500))

In [ ]:
forbeta1_ = pd.merge(forbeta1,corr1,how='left',on='stock_code')
forbeta2_ = pd.merge(forbeta2,corr2,how='left',on='stock_code')
forbeta3_ = pd.merge(forbeta3,corr3,how='left',on='stock_code')
forbeta4_ = pd.merge(forbeta4,corr4,how='left',on='stock_code')
forbeta5_ = pd.merge(forbeta5,corr5,how='left',on='stock_code')
forbeta6_ = pd.merge(forbeta6,corr6,how='left',on='stock_code')

In [ ]:
forbeta1_.rename(columns={0:'corr'},inplace=True)
forbeta2_.rename(columns={0:'corr'},inplace=True)
forbeta3_.rename(columns={0:'corr'},inplace=True)
forbeta4_.rename(columns={0:'corr'},inplace=True)
forbeta5_.rename(columns={0:'corr'},inplace=True)
forbeta6_.rename(columns={0:'corr'},inplace=True)

In [ ]:
forbeta1_ = forbeta1_.drop_duplicates(['stock_code'])
forbeta2_ = forbeta2_.drop_duplicates(['stock_code'])
forbeta3_ = forbeta3_.drop_duplicates(['stock_code'])
forbeta4_ = forbeta4_.drop_duplicates(['stock_code'])
forbeta5_ = forbeta5_.drop_duplicates(['stock_code'])
forbeta6_ = forbeta6_.drop_duplicates(['stock_code'])

In [ ]:
forbeta1_.to_csv('forbeta1_orr.csv',index=False)
forbeta2_.to_csv('forbeta2_orr.csv',index=False)
forbeta3_.to_csv('forbeta3_orr.csv',index=False)
forbeta4_.to_csv('forbeta4_orr.csv',index=False)
forbeta5_.to_csv('forbeta5_orr.csv',index=False)
forbeta6_.to_csv('forbeta6_orr.csv',index=False)

In [ ]:
forbeta1_['beta'] = forbeta1_.apply(lambda x : beta(x['corr'],x['Snp500_std'],x['stock_std']), axis=1)
forbeta2_['beta'] = forbeta2_.apply(lambda x : beta(x['corr'],x['Snp500_std'],x['stock_std']), axis=1)
forbeta3_['beta'] = forbeta3_.apply(lambda x : beta(x['corr'],x['Snp500_std'],x['stock_std']), axis=1)
forbeta4_['beta'] = forbeta4_.apply(lambda x : beta(x['corr'],x['Snp500_std'],x['stock_std']), axis=1)
forbeta5_['beta'] = forbeta5_.apply(lambda x : beta(x['corr'],x['Snp500_std'],x['stock_std']), axis=1)
forbeta6_['beta'] = forbeta6_.apply(lambda x : beta(x['corr'],x['Snp500_std'],x['stock_std']), axis=1)

In [ ]:
# yf로 구한 데이터 베타는 다 구함.
forbeta1_.to_csv('yf_beta_1.csv',index=False)
forbeta2_.to_csv('yf_beta_2.csv',index=False)
forbeta3_.to_csv('yf_beta_3.csv',index=False)
forbeta4_.to_csv('yf_beta_4.csv',index=False)
forbeta5_.to_csv('yf_beta_5.csv',index=False)
forbeta6_.to_csv('yf_beta_6.csv',index=False)

In [ ]:
forbeta1_['bse_ym']='202201'
forbeta2_['bse_ym']='202202'
forbeta3_['bse_ym']='202203'
forbeta4_['bse_ym']='202204'
forbeta5_['bse_ym']='202205'
forbeta6_['bse_ym']='202206'

In [ ]:
yf_beta_01_06 = pd.concat([forbeta1_,forbeta2_,forbeta3_,forbeta4_,forbeta5_,forbeta6_])

In [ ]:
yf_beta_01_06.info()

In [ ]:
yf_beta_01_06 = yf_beta_01_06[['bse_ym','iem_cd','beta']]

In [ ]:
yf_beta_01_06.to_csv('yf_beta_all.csv',index=False)

In [59]:
yf_beta_all = pd.read_csv('yf_beta_all.csv')

In [62]:
kr_fdr_beta_all = pd.read_csv('kr_fdr_beta_all.csv')

In [64]:
yf_beta_all

,bse_ym,stock_code,iem_cd,cur_cd,beta
0,202201,603605.SS,CNE100002TP9,CNY,0.852084
1,202201,STLA.MI,NL00150001Q9,USD,0.852084
2,202201,RACE,NL0011585146,USD,0.852084
3,202201,SVFAU,KYG8601L1106,USD,0.852084
4,202201,LEV.TO,CA5362211040,USD,0.852084
...,...,...,...,...,...
7243,202206,MF,US60500F1057,USD,1.294073
7244,202206,FNCH,US31773D1019,USD,1.294073
7245,202206,MULN,US62526P1093,USD,1.294073
7246,202206,IONQ,US46222L1089,USD,1.294073


In [65]:
kr_fdr_beta_all

,bse_ym,stock_code,beta
0,202201,A287320,1.038303
1,202201,A252720,0.799408
2,202201,A252730,0.839871
3,202201,A334700,-0.083188
4,202201,A301400,0.998643
...,...,...,...
19010,202206,A051910,1.575415
19011,202206,A000660,1.193967
19012,202206,A207940,0.519727
19013,202206,A373220,1.025824


- 국내/ 해외 주식 합치기

In [68]:
All_beta = pd.concat([kr_fdr_beta_all, yf_beta_all])

In [69]:
All_beta = pd.read_csv('All_beta.csv')

#### 포트폴리오 베타 구하기

In [71]:
# 미리 구해놓은 계좌 당 종목별 (금액 * 개수) 가중치 데이터.
bnc_dist = pd.read_csv('bnc_dist.csv')

In [72]:
bnc_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366797 entries, 0 to 366796
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   act_no   366797 non-null  object 
 1   bse_ym   366797 non-null  int64  
 2   iem_cd   366797 non-null  object 
 3   tot_amt  366797 non-null  float64
 4   dist     366797 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 14.0+ MB


In [73]:
merge_bnc = pd.merge(bnc_dist,All_beta,how='left',on=['bse_ym','iem_cd'])

In [74]:
merge_bnc.beta = merge_bnc.beta.fillna(0)

In [ ]:
merge_bnc['distbeta'] = merge_bnc.dist * merge_bnc.beta

In [ ]:
merge_bnc['portforlio_beta'] = merge_bnc.groupby(['bse_ym','act_no'])['distbeta'].transform('sum')

In [ ]:
portfolio_beta = pd.DataFrame(merge_bnc.groupby(['bse_ym','act_no'])['distbeta'].sum())

In [ ]:
portfolio_beta.rename(columns = {'distbeta' : 'portfoliobeta'},inplace=True)

In [ ]:
portfolio_beta.reset_index(inplace=True)

In [ ]:
portfolio_beta.bse_ym.value_counts()

In [ ]:
# 포트폴리오 베타 파일로 저장.
portfolio_beta.to_csv('portfolio_beta_01_06.csv',index=False)

### 2.월평균 일일 거래 빈도

In [ ]:
kr_ord = kr[['act_no', 'orr_dt', 'orr_ord']]
oss_ord = oss[['act_no', 'orr_dt', 'orr_ord']]

In [ ]:
kr_ord_act = kr_ord.groupby(['act_no', 'orr_dt'])[['orr_ord']].max()
kr_ord_act.reset_index(inplace = True)
kr_ord_act

In [ ]:
oss_ord_act = oss_ord.groupby(['act_no', 'orr_dt'])[['orr_ord']].max()
oss_ord_act.reset_index(inplace = True)
oss_ord_act

In [ ]:
ord = pd.concat([kr_ord_act, oss_ord_act])

In [ ]:
# 국내 일일 거래건수와 해외 일일 거래 건수 sum
ord = ord.groupby(['act_no', 'orr_dt'])[['orr_ord']].sum()
ord.reset_index(inplace = True)
ord

In [ ]:
ord.info()

In [ ]:
ord['bse_ym'] = ord['orr_dt'].astype('string').str[:6]
ord.head()

In [ ]:
ord_mean = ord.groupby(['act_no', 'bse_ym'])[['orr_ord']].mean().round(1)
ord_mean.reset_index(inplace = True)

# 월별 일평균 거래빈도
ord_mean

In [ ]:
sns.distplot(ord_mean['orr_ord'])
plt.xlim(-2, 50)

In [ ]:
ord_mean.to_csv('일일거래평균.csv', index = False)

### 3.우량주 투자 비율

#### 우량주 유무 변수 만들기
- 국내, 미국 주식 기준
- 우량주 기준 : 시가총액 상위 100등 이내

In [ ]:
# max columns 재설정
pd.set_option('display.max_columns', 500)

In [ ]:
# 원본 데이터 손실 방지를 위한 copy
info = info.copy()
kr = kr.copy()
oss = oss.copy()
ast = ast.copy()
bnc = bnc.copy()
GA = GA.copy()

# bnc 내 'iem_cd' 공백 제거
bnc_marketcap = bnc[['act_no', 'bse_ym' ,'iem_cd']]
bnc_marketcap['iem_cd'] = bnc_marketcap['iem_cd'].str.strip()
bnc_marketcap['bse_ym'] = bnc_marketcap['bse_ym'].astype('string')
bnc_marketcap

In [ ]:
kr_marketcap = pd.read_csv('주식시세데이터(211201-220630).csv')

# 필요한 칼럼만 추출
kr_marketcap = kr_marketcap[['기준일자', '단축코드', '종목명', '시가총액']]
kr_marketcap = kr_marketcap[(kr_marketcap['기준일자'] == 20220128) | (kr_marketcap['기준일자'] == 20220228) | (kr_marketcap['기준일자'] == 20220331) | (kr_marketcap['기준일자'] == 20220429) | (kr_marketcap['기준일자'] == 20220531) | (kr_marketcap['기준일자'] == 20220630)]
kr_marketcap['기준일자'] = kr_marketcap['기준일자'].astype('string').str[:6]

# 월말 기준 시가총액 상위 100개 종목 추출
kr_marketcap100 = kr_marketcap.sort_values(by = '시가총액', ascending = False).groupby(['기준일자']).head(100)
kr_marketcap100 = kr_marketcap100.sort_values(by = '기준일자')
kr_marketcap100.reset_index(inplace = True)
kr_marketcap100 = kr_marketcap100.drop(columns = 'index')
kr_marketcap100 = kr_marketcap100[['기준일자', '단축코드']]
kr_marketcap100['단축코드'] = 'A' + kr_marketcap100['단축코드']
kr_marketcap100 = kr_marketcap100.rename(columns = {'기준일자' : 'bse_ym', '단축코드' : 'iem_cd'})
kr_marketcap100['우량주 유무'] = 1
kr_marketcap100

In [ ]:
kr_marketcap100.info()

In [ ]:
kr_marketcap100.to_csv('국내주식시가총액100위.csv')

In [78]:
kr_marketcap100 = pd.read_csv('국내주식시가총액100위.csv')

In [ ]:
us_marketcap1 = pd.read_csv('미국시가총액1.csv')
us_marketcap2 = pd.read_csv('미국시가총액2.csv')

us_marketcap100 = pd.concat([us_marketcap1, us_marketcap2])
us_marketcap100.reset_index(inplace = True)
us_marketcap100 = us_marketcap100.drop(columns = 'index')
us_marketcap100 = us_marketcap100[['기호', '시가 총액']]
us_marketcap100 = us_marketcap100.rename(columns = {'기호' : 'symbol'})
us_marketcap100

In [ ]:
us_marketcap100.to_csv('미국주식시가총액100위.csv')

In [79]:
us_marketcap100 = pd.read_csv('미국주식시가총액100위.csv')

In [80]:
oss_ticker = pd.read_csv('oss_isin_ticker.csv')
oss_ticker.head()

,iem_cd,cur_cd,symbol
0,US5494981039,USD,LCID
1,US46222L1089,USD,IONQ
2,US62526P1093,USD,MULN
3,US31773D1019,USD,FNCH
4,US60500F1057,USD,MF


In [81]:
# symbol 과 iem_cd 매칭
us_marketcap100 = us_marketcap100.merge(oss_ticker, how = 'left', on = 'symbol')
us_marketcap100 = us_marketcap100[['iem_cd']]
us_marketcap100['우량주 유무'] = 1
us_marketcap100.head()

,iem_cd,우량주 유무
0,US0378331005,1
1,US5949181045,1
2,US02079K1079,1
3,US02079K3059,1
4,US0231351067,1


In [82]:
# 미국 시가총액 top 100인 우량주가 있으면 1, 없으면 0
bnc_marketcap1 = bnc_marketcap.merge(us_marketcap100, how = 'left', on = 'iem_cd')
bnc_marketcap1 = bnc_marketcap1.fillna(0)

<class 'NameError'>: name 'bnc_marketcap' is not defined

In [ ]:
bnc_marketcap1['우량주 유무'].value_counts()

In [ ]:
# 국내 시가총액 top 100인 우량주가 있으면 1, 없으면 0
bnc_marketcap2 = bnc_marketcap1.merge(kr_marketcap100, how = 'left', on = ['bse_ym', 'iem_cd'])
bnc_marketcap2 = bnc_marketcap2.fillna(0)
bnc_marketcap2
bnc_marketcap2['우량주 유무'] = bnc_marketcap2['우량주 유무_x'] + bnc_marketcap2['우량주 유무_y']

bnc_marketcap = bnc_marketcap2[['act_no', 'bse_ym', 'iem_cd', '우량주 유무']] #필요한 칼럼만 추출
bnc_marketcap

In [ ]:
bnc_marketcap2['우량주 유무_x'].value_counts()

In [ ]:
bnc_marketcap.to_csv('시가총액변수.csv')

In [84]:
bnc_marketcap = pd.read_csv('시가총액변수.csv')

In [85]:
bnc_dist = pd.read_csv('bnc_dist.csv')

In [86]:
bnc_dist['iem_cd'] = bnc_dist['iem_cd'].str.strip()
bnc_dist['bse_ym'] = bnc_dist['bse_ym'].astype('string')
bnc_dist

,act_no,bse_ym,iem_cd,tot_amt,dist
0,0000b53f15d9484a62a745bdb02cd5f27796ba07c1bddc...,202201,A391060,58100.0,0.268677
1,0000b53f15d9484a62a745bdb02cd5f27796ba07c1bddc...,202201,A391710,39185.0,0.181207
2,0000b53f15d9484a62a745bdb02cd5f27796ba07c1bddc...,202201,A396770,63300.0,0.292724
3,0000b53f15d9484a62a745bdb02cd5f27796ba07c1bddc...,202201,A400760,55660.0,0.257393
4,0000b53f15d9484a62a745bdb02cd5f27796ba07c1bddc...,202202,A377330,25000.0,0.102883
...,...,...,...,...,...
366792,ffff314cc371dd13571dbde2fc9355e142b8e88d4b9790...,202203,A27957K,0.0,0.000000
366793,ffff314cc371dd13571dbde2fc9355e142b8e88d4b9790...,202204,A27957K,0.0,0.000000
366794,ffff314cc371dd13571dbde2fc9355e142b8e88d4b9790...,202205,A27957K,0.0,0.000000
366795,ffff314cc371dd13571dbde2fc9355e142b8e88d4b9790...,202206,A279570,0.0,0.000000


In [ ]:
bnc_marcap = bnc_dist.merge(bnc_marketcap, how = 'left', on = ['act_no', 'bse_ym', 'iem_cd' ])
bnc_marcap['우량주 비율'] = bnc_marcap['dist'] * bnc_marcap['우량주 유무']
bnc_marcap = bnc_marcap.groupby(['act_no', 'bse_ym'])[['우량주 비율']].sum().round(3)
bnc_marcap.reset_index(inplace = True)
bnc_marcap

In [ ]:
bnc_marcap['우량주 비율'].value_counts()

In [ ]:
bnc_marcap.to_csv('우량주 비율변수.csv', index = False)

### 4.파생상품 보유 개수 (국내만)

#### 파생상품 유무 변수 만들기
- 파생상품이 존재 : 1, 존재하지 않음 : 0

In [ ]:
bnc_kr = bnc[['act_no', 'bse_ym' , 'iem_cd', 'cur_cd']]
bnc_kr = bnc_kr[bnc_kr['cur_cd'] == 'KRW']
bnc_kr['파생상품 유무'] = bnc_kr['iem_cd'].str[0] # 종목코드가 A로 시작하면 국내주식, 아니면 파생상품

# 파생상품이 있으면 '1' 부여, 없으면 '0' 부여
bnc_kr['파생상품 유무'] = bnc_kr['파생상품 유무'].apply(lambda x : 0 if x == 'A' else 1)
bnc_kr.head()

In [ ]:
bnc_kr = bnc_kr[['act_no', 'bse_ym', '파생상품 유무']]
bnc_kr = bnc_kr.groupby(['act_no', 'bse_ym'])[['파생상품 유무']].sum()
bnc_kr.reset_index(inplace = True)
bnc_kr

In [ ]:
bnc_kr.to_csv('파생상품 유무.csv')

### 5.정보 수집 성향 지수

#### 정보 수집 성향 변수 만들기
- GA 데이터를 이용한 투자자의 투자 정보 수집 성향 파악
- 주식 현재가 SCREEN 의 EVENTACTION 유형(11개)을 이용
- 총 11가지의 투자 정보 중에서 몇 개를 고려하는지를 파악

In [ ]:
# 현재가_상세 screen
search = GA[GA['EVENTCATEGORY'] == '현재가_상세']
search[['EVENTACTION']].value_counts()

In [ ]:
sns.countplot(x = search_cnt['EVENTACTION'], data = search_cnt)
plt.title("고객 한 명이 고려하는 정보 유형의 수")
plt.xlabel("Total Count of Eventaction per customer")
plt.tight_layout()

In [ ]:
search_cnt = search.groupby(['CUS_NO'])[['EVENTACTION']].nunique()
search_cnt.reset_index(inplace = True)
search_cnt
search_cnt.to_csv('정보수집성향.csv')

## 분류모델링

### 분류 모델링을 위한 데이터 프레임 결합하기

In [12]:
beta = pd.read_csv('portfolio_beta_01_06.csv')
ord_mean = pd.read_csv('일일거래평균.csv')
ord_mean = ord_mean[['act_no' ,'bse_ym', 'orr_ord']]
marcap = pd.read_csv('우량주 비율변수.csv')
bnc_kr = pd.read_csv('파생상품 유무.csv')
search = pd.read_csv('정보수집성향.csv')

In [13]:
bnc

,act_no,bse_ym,iem_cd,fc_sec_trd_nat_cd,itg_pdt_tp_cd,stl_bse_itg_bnc_qty,itg_byn_cns_qty,itg_sll_cns_qty,cns_bse_itg_bnc_qty,stl_bse_now_eal_amt,stl_bse_fc_now_eal_amt,cns_bse_now_eal_amt,cns_bse_fc_now_eal_amt,lon_amt,cur_cd
0,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,202205,A101240,0,10,14.0,0.0,0.0,14.0,101640.0,0.0,101640.0,0.0,0.0,KRW
1,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,202205,A900120,0,10,341.0,0.0,341.0,0.0,87978.0,0.0,0.0,0.0,0.0,KRW
2,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,202205,A024910,0,10,32.0,0.0,32.0,0.0,77280.0,0.0,0.0,0.0,0.0,KRW
3,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,202204,A263020,0,10,52.0,0.0,0.0,52.0,184600.0,0.0,184600.0,0.0,0.0,KRW
4,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,202204,A037030,0,10,18.0,0.0,0.0,18.0,89010.0,0.0,89010.0,0.0,0.0,KRW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366792,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,202203,A005930,0,10,2.0,0.0,0.0,2.0,139200.0,0.0,139200.0,0.0,0.0,KRW
366793,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,202205,A353590,0,10,2.0,0.0,0.0,2.0,29400.0,0.0,29400.0,0.0,0.0,KRW
366794,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,202204,A353590,0,10,2.0,0.0,0.0,2.0,30200.0,0.0,30200.0,0.0,0.0,KRW
366795,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,202203,A353590,0,10,0.0,2.0,0.0,2.0,0.0,0.0,48100.0,0.0,0.0,KRW


In [14]:
cus = info[['cus_no', 'bse_ym','act_no']]
cus

,cus_no,bse_ym,act_no
0,92ed7cdc3e3673226a613892c47a8eccfe602d0fbdb349...,202203,3610f0105b67e30709e55c546c75c45b8c9d589e90fb9f...
1,92ed7cdc3e3673226a613892c47a8eccfe602d0fbdb349...,202202,3610f0105b67e30709e55c546c75c45b8c9d589e90fb9f...
2,92ed7cdc3e3673226a613892c47a8eccfe602d0fbdb349...,202206,3610f0105b67e30709e55c546c75c45b8c9d589e90fb9f...
3,92ed7cdc3e3673226a613892c47a8eccfe602d0fbdb349...,202205,3610f0105b67e30709e55c546c75c45b8c9d589e90fb9f...
4,92ed7cdc3e3673226a613892c47a8eccfe602d0fbdb349...,202201,3610f0105b67e30709e55c546c75c45b8c9d589e90fb9f...
...,...,...,...
59076,1b3dc5053fa9c52990872c97f85d54f3e8749eabf64b5d...,202204,b41f38d7eedb84857fcc8fe2f58ded96aa3bbd85614d18...
59077,1b3dc5053fa9c52990872c97f85d54f3e8749eabf64b5d...,202202,b41f38d7eedb84857fcc8fe2f58ded96aa3bbd85614d18...
59078,1b3dc5053fa9c52990872c97f85d54f3e8749eabf64b5d...,202206,b41f38d7eedb84857fcc8fe2f58ded96aa3bbd85614d18...
59079,1b3dc5053fa9c52990872c97f85d54f3e8749eabf64b5d...,202201,b41f38d7eedb84857fcc8fe2f58ded96aa3bbd85614d18...


In [17]:
total = bnc.merge(cus, how = 'left', on = ['act_no', 'bse_ym'])
total = total[['act_no', 'cus_no', 'bse_ym']]
total

,act_no,cus_no,bse_ym
0,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202205
1,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202205
2,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202205
3,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202204
4,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202204
...,...,...,...
366792,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202203
366793,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202205
366794,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202204
366795,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202203


In [18]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366797 entries, 0 to 366796
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   act_no  366797 non-null  object
 1   cus_no  366797 non-null  object
 2   bse_ym  366797 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 11.2+ MB


In [19]:
m1 = total.merge(search, how = 'left', on = ['cus_no', 'bse_ym'])
m1 = m1.fillna(0)
m2 = m1.merge(beta, how = 'left', on = ['act_no' , 'bse_ym'])
m2
m3 = m2.merge(bnc_kr, how = 'left', on = ['act_no', 'bse_ym'])
m3
m4 = m3.merge(marcap, how = 'left' , on = ['act_no', 'bse_ym'])
m4
m5 = m4.merge(ord_mean, how = 'left', on = ['act_no' ,'bse_ym'])
m5 = m5.fillna(0)
m5

,act_no,cus_no,bse_ym,EVENTACTION,portfoliobeta,파생상품 유무,우량주 비율,orr_ord
0,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202205,4.0,0.822003,3.0,0.005,24.0
1,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202205,4.0,0.822003,3.0,0.005,24.0
2,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202205,4.0,0.822003,3.0,0.005,24.0
3,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202204,5.0,1.059392,1.0,0.006,11.7
4,e97920ce7d2205d876398178d56a854ebe6aa84f46a9ff...,89a691f66d34b51a635fdcaa93e0142f93b511855e6599...,202204,5.0,1.059392,1.0,0.006,11.7
...,...,...,...,...,...,...,...,...
366792,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202203,0.0,0.771525,0.0,1.000,1.0
366793,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202205,0.0,0.764160,0.0,0.821,0.0
366794,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202204,0.0,0.845233,0.0,0.817,0.0
366795,8a302c32df0a3110af9fea8285cb720cfba9cd57b87123...,6e5e1594c104dc8b9682e2a194fa8fd7eea167f4cf2a07...,202203,0.0,0.771525,0.0,1.000,1.0


In [20]:
final = m5
final.to_csv('최종데이터파일.csv', index = False)

#### - 변수간 상관계수 파악

In [ ]:
final[['EVENTACTION', 'portfoliobeta', '파생상품 유무','우량주 비율', 'orr_ord']].corr()

####  - 결측치 유무 확인

In [ ]:
final.isna().sum()

In [ ]:
# 파생상품을 개수로 count했지만, 상품 보유여부로 바꿈.
final = final.rename(columns = {'EVENTACTION' : '정보수집지수' , 'portfoliobeta' : '베타계수', 'orr_ord' : '일일 거래평균'})
final.loc[final['파생상품 유무'] > 0, '파생상품 유무'] = 1
final

#### - 변수 타입 확인

In [ ]:
final_scaled = final[['정보수집지수' , '베타계수', '파생상품 유무', '우량주 비율', '일일 거래평균']]

#### - 변수 스케일링 

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(final_scaled)
final_scaled = scaler.transform(final_scaled)

final_scaled = pd.DataFrame(final_scaled, columns = ['정보수집지수' , '베타계수', '파생상품 유무', '우량주 비율', '일일 거래평균'])
final_scaled

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from sklearn.cluster import KMeans

model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,10))
visualizer.fit(final_scaled)

In [ ]:
k = 6

# 그룹 수, random_state 설정
model = KMeans(n_clusters = k, random_state = 10)

# 정규화된 데이터에 학습
model.fit(final_scaled)

# 클러스터링 결과 각 데이터가 몇 번째 그룹에 속하는지 저장
final['cluster_id'] = model.fit_predict(final_scaled)

In [ ]:
final.head()

In [ ]:
final['cluster_id'].value_counts()

In [ ]:
sns.boxplot(y='베타계수',x='cluster_id',data= final)

In [ ]:
sns.boxplot(y='우량주 비율',x='cluster_id',data= final)

In [ ]:
sns.boxplot(y='정보수집지수',x='cluster_id',data= final)

In [ ]:
sns.boxplot(y='파생상품 유무',x='cluster_id',data= final)

In [ ]:
sns.boxplot(y='일일 거래평균',x='cluster_id',data= final)

In [ ]:
final_scaled['cluster_id'] = final['cluster_id']
final_scaled['act_no'] = final['act_no']
final_scaled

In [ ]:
df_melt = pd.melt(final_scaled.reset_index(), id_vars = ['act_no', 'cluster_id'], value_vars = ['베타계수' ,'정보수집지수', '파생상품 유무', '일일 거래평균', '우량주 비율'], var_name = 'Metric', value_name = 'Value')
plt.xlabel('Metirc')
sns.pointplot(data = df_melt, x = 'Metric', y = 'Value', hue = 'cluster_id')

In [ ]:
c0 = final[final['cluster_id']==0][['act_no','cus_no','bse_ym']]
c1 = final[final['cluster_id']==1][['act_no','cus_no','bse_ym']]
c2 = final[final['cluster_id']==2][['act_no','cus_no','bse_ym']]
c3 = final[final['cluster_id']==3][['act_no','cus_no','bse_ym']]
c4 = final[final['cluster_id']==4][['act_no','cus_no','bse_ym']]
c5 = final[final['cluster_id']==5][['act_no','cus_no','bse_ym']]

In [ ]:
c0 = pd.merge(c0,info,how='left',on=['act_no','cus_no','bse_ym'])
c1 = pd.merge(c1,info,how='left',on=['act_no','cus_no','bse_ym'])
c2 = pd.merge(c2,info,how='left',on=['act_no','cus_no','bse_ym'])
c3 = pd.merge(c3,info,how='left',on=['act_no','cus_no','bse_ym'])
c4 = pd.merge(c4,info,how='left',on=['act_no','cus_no','bse_ym'])
c5 = pd.merge(c5,info,how='left',on=['act_no','cus_no','bse_ym'])

In [ ]:
# 클러스터별 라이프스테이지 세그먼트 분포 확인
plt.figure(figsize = (16,10))
plt.subplot(3,3,1)
plt.title('cluster_id = 0')
sns.countplot( x = 'lsg_sgm_cd' ,data = c0, order = c0.lsg_sgm_cd.value_counts().index)
              
plt.subplot(3,3,2)
plt.title('cluster_id = 1')
sns.countplot( x = 'lsg_sgm_cd' ,data = c1, order = c1.lsg_sgm_cd.value_counts().index)

plt.subplot(3,3,3)
plt.title('cluster_id = 2')
sns.countplot( x = 'lsg_sgm_cd' ,data = c2, order = c2.lsg_sgm_cd.value_counts().index)

plt.subplot(3,3,4)
plt.title('cluster_id = 3')
sns.countplot( x = 'lsg_sgm_cd' ,data = c3, order = c3.lsg_sgm_cd.value_counts().index)

plt.subplot(3,3,5)
plt.title('cluster_id = 4')
sns.countplot( x = 'lsg_sgm_cd' ,data = c4, order = c4.lsg_sgm_cd.value_counts().index)

plt.subplot(3,3,6)
plt.title('cluster_id = 5')
sns.countplot( x = 'lsg_sgm_cd' ,data = c5, order = c5.lsg_sgm_cd.value_counts().index)

plt.tight_layout()

In [ ]:
# 클러스터별 주거래 시장구분코드 분포 확인
plt.figure(figsize = (16,10))
plt.subplot(3,3,1)
plt.title('cluster_id = 0')
sns.countplot( x = 'mrz_mkt_dit_cd' ,data = c0, order = c0.mrz_mkt_dit_cd.value_counts().index)
              
plt.subplot(3,3,2)
plt.title('cluster_id = 1')
sns.countplot( x = 'mrz_mkt_dit_cd' ,data = c1, order = c1.mrz_mkt_dit_cd.value_counts().index)

plt.subplot(3,3,3)
plt.title('cluster_id = 2')
sns.countplot( x = 'mrz_mkt_dit_cd' ,data = c2, order = c2.mrz_mkt_dit_cd.value_counts().index)

plt.subplot(3,3,4)
plt.title('cluster_id = 3')
sns.countplot( x = 'mrz_mkt_dit_cd' ,data = c3, order = c3.mrz_mkt_dit_cd.value_counts().index)

plt.subplot(3,3,5)
plt.title('cluster_id = 4')
sns.countplot( x = 'mrz_mkt_dit_cd' ,data = c4, order = c4.mrz_mkt_dit_cd.value_counts().index)

plt.subplot(3,3,6)
plt.title('cluster_id = 5')
sns.countplot( x = 'mrz_mkt_dit_cd' ,data = c5, order = c5.mrz_mkt_dit_cd.value_counts().index)

plt.tight_layout()

In [ ]:
# 클러스터별 주거래 업종 분포 확인
plt.figure(figsize = (16,10))
plt.subplot(3,3,1)
plt.title('cluster_id = 0')
sns.countplot( x = 'mrz_btp_dit_cd' ,data = c0, order = c0.mrz_btp_dit_cd.value_counts().index)
              
plt.subplot(3,3,2)
plt.title('cluster_id = 1')
sns.countplot( x = 'mrz_btp_dit_cd' ,data = c1, order = c1.mrz_btp_dit_cd.value_counts().index)

plt.subplot(3,3,3)
plt.title('cluster_id = 2')
sns.countplot( x = 'mrz_btp_dit_cd' ,data = c2, order = c2.mrz_btp_dit_cd.value_counts().index)

plt.subplot(3,3,4)
plt.title('cluster_id = 3')
sns.countplot( x = 'mrz_btp_dit_cd' ,data = c3, order = c3.mrz_btp_dit_cd.value_counts().index)

plt.subplot(3,3,5)
plt.title('cluster_id = 4')
sns.countplot( x = 'mrz_btp_dit_cd' ,data = c4, order = c4.mrz_btp_dit_cd.value_counts().index)

plt.subplot(3,3,6)
plt.title('cluster_id = 5')
sns.countplot( x = 'mrz_btp_dit_cd' ,data = c5, order = c5.mrz_btp_dit_cd.value_counts().index)

plt.tight_layout()

In [ ]:
# 클러스터별 고겍자산구간 분포 확인
plt.figure(figsize = (16,10))
plt.subplot(3,3,1)
plt.title('cluster_id = 0')
sns.countplot( x = 'cus_aet_stn_cd' ,data = c0, order = c0.cus_aet_stn_cd.value_counts().index)
              
plt.subplot(3,3,2)
plt.title('cluster_id = 1')
sns.countplot( x = 'cus_aet_stn_cd' ,data = c1, order = c1.cus_aet_stn_cd.value_counts().index)

plt.subplot(3,3,3)
plt.title('cluster_id = 2')
sns.countplot( x = 'cus_aet_stn_cd' ,data = c2, order = c2.cus_aet_stn_cd.value_counts().index)

plt.subplot(3,3,4)
plt.title('cluster_id = 3')
sns.countplot( x = 'cus_aet_stn_cd' ,data = c3, order = c3.cus_aet_stn_cd.value_counts().index)

plt.subplot(3,3,5)
plt.title('cluster_id = 4')
sns.countplot( x = 'cus_aet_stn_cd' ,data = c4, order = c4.cus_aet_stn_cd.value_counts().index)

plt.subplot(3,3,6)
plt.title('cluster_id = 5')
sns.countplot( x = 'cus_aet_stn_cd' ,data = c5, order = c5.cus_aet_stn_cd.value_counts().index)

plt.tight_layout()

In [ ]:
# 클러스터별 고객 수익금액구간 분포 확인
plt.figure(figsize = (16,10))
plt.subplot(3,3,1)
plt.title('cluster_id = 0')
sns.countplot( x = 'pft_amt_stn_cd' ,data = c0, order = c0.pft_amt_stn_cd.value_counts().index)
              
plt.subplot(3,3,2)
plt.title('cluster_id = 1')
sns.countplot( x = 'pft_amt_stn_cd' ,data = c1, order = c1.pft_amt_stn_cd.value_counts().index)

plt.subplot(3,3,3)
plt.title('cluster_id = 2')
sns.countplot( x = 'pft_amt_stn_cd' ,data = c2, order = c2.pft_amt_stn_cd.value_counts().index)

plt.subplot(3,3,4)
plt.title('cluster_id = 3')
sns.countplot( x = 'pft_amt_stn_cd' ,data = c3, order = c3.pft_amt_stn_cd.value_counts().index)

plt.subplot(3,3,5)
plt.title('cluster_id = 4')
sns.countplot( x = 'pft_amt_stn_cd' ,data = c4, order = c4.pft_amt_stn_cd.value_counts().index)

plt.subplot(3,3,6)
plt.title('cluster_id = 5')
sns.countplot( x = 'pft_amt_stn_cd' ,data = c5, order = c5.pft_amt_stn_cd.value_counts().index)

plt.tight_layout()

In [ ]:
# 클러스터별 총 투자기간 세그먼트 분포 확인
plt.figure(figsize = (16,10))
plt.subplot(3,3,1)
plt.title('cluster_id = 0')
sns.countplot( x = 'tot_ivs_te_sgm_cd' ,data = c0, order = c0.tot_ivs_te_sgm_cd.value_counts().index)
              
plt.subplot(3,3,2)
plt.title('cluster_id = 1')
sns.countplot( x = 'tot_ivs_te_sgm_cd' ,data = c1, order = c1.tot_ivs_te_sgm_cd.value_counts().index)

plt.subplot(3,3,3)
plt.title('cluster_id = 2')
sns.countplot( x = 'tot_ivs_te_sgm_cd' ,data = c2, order = c2.tot_ivs_te_sgm_cd.value_counts().index)

plt.subplot(3,3,4)
plt.title('cluster_id = 3')
sns.countplot( x = 'tot_ivs_te_sgm_cd' ,data = c3, order = c3.tot_ivs_te_sgm_cd.value_counts().index)

plt.subplot(3,3,5)
plt.title('cluster_id = 4')
sns.countplot( x = 'tot_ivs_te_sgm_cd' ,data = c4, order = c4.tot_ivs_te_sgm_cd.value_counts().index)

plt.subplot(3,3,6)
plt.title('cluster_id = 5')
sns.countplot( x = 'tot_ivs_te_sgm_cd' ,data = c5, order = c5.tot_ivs_te_sgm_cd.value_counts().index)

plt.tight_layout()